In [1]:
import os
import numpy as np 
import pandas as pd 
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
"import plotly.express as px"
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
from sklearn import preprocessing
import matplotlib 
matplotlib.style.use('ggplot')
from sklearn.preprocessing import LabelEncoder

In [23]:
df=pd.read_csv("./heart.csv")
df.head()
string_col = df.select_dtypes(include="object").columns
df[string_col]=df[string_col].astype("string")
df.dtypes
# df_tree=pd.get_dummies(df,columns=string_col,drop_first=False)
# df_tree.head()

df_tree = df.apply(LabelEncoder().fit_transform)
df_tree.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,12,1,1,41,147,0,1,98,0,10,2,0
1,21,0,2,55,40,0,1,82,0,20,1,1
2,9,1,1,31,141,0,2,25,0,10,2,0
3,20,0,0,39,72,0,1,34,1,25,1,1
4,26,1,2,49,53,0,1,48,0,10,2,0


Pour utiliser un autre dataset, il suffit de changer le chemin. Ici, on calcule la taille de l'effet. C'est une mesure qui quantifie l'ampleur de la différence entre deux groupes indépendant sur une variable continue. On va chercher à calculer à quel point la moyenne de chaque facteur diffère entre les patients ayant une maladie cardiaque et ceux qui n'en ont pas (en prenant en compte la variabilité).

In [24]:
def cohens_d(group1, group2):
    mean1, mean2 = np.mean(group1), np.mean(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    n1, n2 = len(group1), len(group2)
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    
    d = abs(mean1 - mean2) / pooled_std
    return d

target = df.columns[-1]
X = df.drop(target, axis=1)
y = df[target]

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

effect_sizes = {}
for col in numeric_cols:
    group0 = df[df[target] == 0][col].values
    group1 = df[df[target] == 1][col].values
    effect_sizes[col] = cohens_d(group0, group1)

effect_sizes_df = pd.DataFrame(list(effect_sizes.items()), columns=['Facteur', "Taille d'effet"])
effect_sizes_df = effect_sizes_df.sort_values("Taille d'effet", ascending=False)

print("Tailles d'effet selon l'échelle de Cohen:")
print(effect_sizes_df)
print(f"Taille d'effet moyenne: {effect_sizes_df["Taille d'effet"].mean():.4f}")

Tailles d'effet selon l'échelle de Cohen:
       Facteur  Taille d'effet
5      Oldpeak        0.887272
4        MaxHR        0.878030
0          Age        0.590680
3    FastingBS        0.557347
2  Cholesterol        0.480851
1    RestingBP        0.217441
Taille d'effet moyenne: 0.6019
